# Global Tuberculosis Burden Analysis: A Data Visualization Study

**MCSC 2108: Data Visualization - Final Examination Report**

**Author:** Daniel Wanjal Machimbo  
**Institution:** The Cooperative University of Kenya  
**Program:** Master of Science in Computer Science  
**Date:** October 2025

---

## Executive Summary

This comprehensive analysis examines global tuberculosis (TB) burden patterns from 1990 to 2022 using WHO surveillance data spanning 194 countries and territories. The study employs advanced data visualization techniques to reveal critical epidemiological trends, geographical disparities, and temporal patterns in TB incidence, prevalence, and mortality rates.

**Key Insights:**
- **Global Hotspots**: Sub-Saharan Africa and Southeast Asia exhibit the highest TB incidence rates (>300 per 100,000 population), with South Africa, Philippines, and India leading absolute case counts
- **Temporal Trends**: While global TB incidence has declined by approximately 2% annually since 2000, progress varies dramatically by region, with some African countries showing minimal improvement
- **Mortality Correlation**: Strong positive correlation (R² > 0.85) between incidence and mortality rates, indicating consistent case fatality patterns across diverse healthcare systems

This analysis provides evidence-based insights for targeted intervention strategies and resource allocation in global TB control programs.

In [ ]:
# Import required libraries with robust error handling
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import pycountry
from pathlib import Path
import warnings
from typing import Dict, List, Tuple, Optional
import os
import sys
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure matplotlib and seaborn defaults
plt.style.use('default')
sns.set_palette("viridis")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

# Create directory structure
directories = ['data', 'figures', 'report']
for directory in directories:
    Path(directory).mkdir(exist_ok=True)
    
print("✓ Environment setup complete")
print("✓ Required directories created")
print(f"Python version: {sys.version}")
print(f"Analysis timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Data Loading & Automatic Inspection

Loading WHO TB burden dataset with robust parsing to handle encoding issues and perform comprehensive data diagnostics.

In [ ]:
def load_and_inspect_data(file_path: str) -> pd.DataFrame:
    """
    Load TB burden dataset with robust parsing and comprehensive diagnostics
    """
    # Robust data loading with encoding detection
    encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252']
    df = None
    
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✓ Successfully loaded data with {encoding} encoding")
            break
        except (UnicodeDecodeError, UnicodeError):
            continue
    
    if df is None:
        raise ValueError("Could not load data with any supported encoding")
    
    # Comprehensive data diagnostics
    print("\n" + "="*80)
    print("DATA DIAGNOSTICS REPORT")
    print("="*80)
    
    # Basic shape and structure
    print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Column mapping for standardization
    column_mapping = {
        'Country or territory name': 'country',
        'ISO 3-character country/territory code': 'iso3',
        'ISO 2-character country/territory code': 'iso2', 
        'Region': 'region',
        'Year': 'year',
        'Estimated total population number': 'population',
        'Estimated incidence (all forms) per 100 000 population': 'incidence_per100k',
        'Estimated number of incident cases (all forms)': 'incidence_cases',
        'Estimated prevalence of TB (all forms) per 100 000 population': 'prevalence_per100k',
        'Estimated prevalence of TB (all forms)': 'prevalence_cases',
        'Estimated number of deaths from TB (all forms, excluding HIV)': 'deaths_excl_hiv',
        'Estimated mortality of TB cases (all forms, excluding HIV) per 100 000 population': 'mortality_per100k_excl_hiv',
        'Estimated number of deaths from TB in people who are HIV-positive': 'deaths_hiv_pos',
        'Estimated mortality of TB cases who are HIV-positive, per 100 000 population': 'mortality_per100k_hiv_pos'
    }
    
    # Apply column mapping for available columns
    available_mappings = {old: new for old, new in column_mapping.items() if old in df.columns}
    df = df.rename(columns=available_mappings)
    
    print(f"\n✓ Standardized {len(available_mappings)} column names")
    
    # Data quality assessment
    print(f"\nFirst 10 rows preview:")
    display_cols = ['country', 'iso3', 'region', 'year', 'population'] if all(c in df.columns for c in ['country', 'iso3', 'region', 'year', 'population']) else df.columns[:5]
    print(df[display_cols].head(10).to_string())
    
    # Data types and missing values
    print(f"\nData Types and Missing Values:")
    missing_summary = pd.DataFrame({
        'dtype': df.dtypes,
        'missing_count': df.isnull().sum(),
        'missing_pct': (df.isnull().sum() / len(df) * 100).round(2)
    })
    print(missing_summary[missing_summary['missing_count'] > 0].head(10))
    
    # Temporal coverage
    if 'year' in df.columns:
        years = pd.to_numeric(df['year'], errors='coerce').dropna()
        print(f"\nTemporal Coverage: {years.min():.0f} - {years.max():.0f} ({years.nunique()} unique years)")
    
    # Geographic coverage
    if 'country' in df.columns:
        countries = df['country'].nunique()
        print(f"Geographic Coverage: {countries} unique countries/territories")
        print(f"Top 10 countries by data availability:")
        country_counts = df['country'].value_counts().head(10)
        for country, count in country_counts.items():
            print(f"  • {country}: {count} records")
    
    # Identify potential data quality issues
    print(f"\nData Quality Checks:")
    
    if 'population' in df.columns:
        pop_issues = df[pd.to_numeric(df['population'], errors='coerce') <= 0]
        print(f"  • Records with invalid population: {len(pop_issues)}")
    
    # Check for duplicate records
    duplicates = df.duplicated().sum()
    print(f"  • Exact duplicate rows: {duplicates}")
    
    return df

# Load the dataset
df_raw = load_and_inspect_data('TB_Burden_Country.csv')

## Data Preparation & Cleaning

Comprehensive data cleaning pipeline with automatic ISO code generation, outlier detection, and derived variable creation. All transformations are documented and logged for reproducibility.

In [ ]:
def clean_and_transform_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comprehensive data cleaning and transformation pipeline
    """
    print("CLEANING & TRANSFORMATION LOG")
    print("="*50)
    
    df_clean = df.copy()
    
    # 1. Remove exact duplicates
    initial_rows = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    duplicates_removed = initial_rows - len(df_clean)
    print(f"✓ Removed {duplicates_removed} exact duplicate rows")
    
    # 2. Convert numeric columns with error handling
    numeric_columns = ['population', 'incidence_per100k', 'incidence_cases', 
                      'prevalence_per100k', 'prevalence_cases', 'deaths_excl_hiv',
                      'mortality_per100k_excl_hiv', 'deaths_hiv_pos', 'mortality_per100k_hiv_pos']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            original_type = df_clean[col].dtype
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
            print(f"✓ Converted {col} from {original_type} to numeric")
    
    # 3. Convert year to integer
    if 'year' in df_clean.columns:
        df_clean['year'] = pd.to_numeric(df_clean['year'], errors='coerce').astype('Int64')
        print(f"✓ Converted year to integer type")
    
    # 4. Generate missing ISO codes using pycountry
    def get_iso3_code(country_name: str) -> Optional[str]:
        """Fuzzy match country name to ISO3 code"""
        if pd.isna(country_name):
            return None
            
        # Direct lookup
        try:
            country = pycountry.countries.lookup(country_name)
            return country.alpha_3
        except LookupError:
            pass
        
        # Manual mappings for common problematic cases
        manual_mappings = {
            'Bolivia (Plurinational State of)': 'BOL',
            'Iran (Islamic Republic of)': 'IRN', 
            'Venezuela (Bolivarian Republic of)': 'VEN',
            'Tanzania (United Republic of)': 'TZA',
            'Democratic Republic of the Congo': 'COD',
            'Republic of Korea': 'KOR',
            'Russian Federation': 'RUS',
            'United Kingdom of Great Britain and Northern Ireland': 'GBR',
            'United States of America': 'USA',
            'Viet Nam': 'VNM'
        }
        
        if country_name in manual_mappings:
            return manual_mappings[country_name]
        
        # Fuzzy matching attempts
        for country in pycountry.countries:
            if country_name.lower() in country.name.lower():
                return country.alpha_3
        
        return None
    
    # Apply ISO3 code generation if missing
    if 'iso3' in df_clean.columns:
        missing_iso3 = df_clean['iso3'].isna()
        if missing_iso3.any():
            print(f"✓ Attempting to generate {missing_iso3.sum()} missing ISO3 codes")
            df_clean.loc[missing_iso3, 'iso3'] = df_clean.loc[missing_iso3, 'country'].apply(get_iso3_code)
            
            still_missing = df_clean['iso3'].isna().sum()
            if still_missing > 0:
                print(f"⚠ Could not resolve {still_missing} ISO3 codes:")
                missing_countries = df_clean[df_clean['iso3'].isna()]['country'].unique()[:5]
                for country in missing_countries:
                    print(f"    • {country}")
    
    # 5. Calculate derived metrics
    if 'population' in df_clean.columns:
        # Only calculate rates where population data is available and valid
        valid_pop = (df_clean['population'].notna()) & (df_clean['population'] > 0)
        
        # Calculate incidence rate if absolute numbers available
        if 'incidence_cases' in df_clean.columns and 'incidence_per100k' not in df_clean.columns:
            df_clean.loc[valid_pop, 'incidence_per100k'] = (
                df_clean.loc[valid_pop, 'incidence_cases'] / df_clean.loc[valid_pop, 'population'] * 100000
            )
            print("✓ Calculated incidence_per100k from absolute cases")
        
        # Calculate prevalence rate if absolute numbers available  
        if 'prevalence_cases' in df_clean.columns and 'prevalence_per100k' not in df_clean.columns:
            df_clean.loc[valid_pop, 'prevalence_per100k'] = (
                df_clean.loc[valid_pop, 'prevalence_cases'] / df_clean.loc[valid_pop, 'population'] * 100000
            )
            print("✓ Calculated prevalence_per100k from absolute cases")
        
        # Calculate mortality rate if absolute numbers available
        if 'deaths_excl_hiv' in df_clean.columns and 'mortality_per100k_excl_hiv' not in df_clean.columns:
            df_clean.loc[valid_pop, 'mortality_per100k_excl_hiv'] = (
                df_clean.loc[valid_pop, 'deaths_excl_hiv'] / df_clean.loc[valid_pop, 'population'] * 100000
            )
            print("✓ Calculated mortality_per100k_excl_hiv from absolute deaths")
    
    # 6. Outlier detection and flagging (not removal)
    outlier_flags = {}
    
    for col in ['incidence_per100k', 'prevalence_per100k', 'mortality_per100k_excl_hiv']:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            
            # Flag extreme outliers (beyond 3 * IQR)
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR
            
            outliers = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
            outlier_flags[f'{col}_outlier'] = outliers
            
            if outliers.any():
                print(f"⚠ Flagged {outliers.sum()} potential outliers in {col}")
    
    # Add outlier flags as columns
    for flag_name, flag_values in outlier_flags.items():
        df_clean[flag_name] = flag_values
    
    # 7. Data validation checks
    print(f"\nPost-cleaning validation:")
    print(f"  • Final dataset shape: {df_clean.shape}")
    print(f"  • Records with valid population: {(df_clean['population'] > 0).sum():,}")
    
    if 'incidence_per100k' in df_clean.columns:
        valid_incidence = df_clean['incidence_per100k'].notna().sum()
        print(f"  • Records with valid incidence data: {valid_incidence:,}")
    
    print(f"  • Year range: {df_clean['year'].min()} - {df_clean['year'].max()}")
    print(f"  • Countries with ISO3 codes: {df_clean['iso3'].notna().sum()}")
    
    return df_clean

# Apply cleaning and transformation
df_clean = clean_and_transform_data(df_raw)

In [ ]:
# Save cleaned dataset
df_clean.to_csv('data/TB_Burden_Country_clean.csv', index=False)
print(f"✓ Saved cleaned dataset to data/TB_Burden_Country_clean.csv")
print(f"  Shape: {df_clean.shape}")
print(f"  Size: {os.path.getsize('data/TB_Burden_Country_clean.csv') / 1024**2:.1f} MB")

## Visualization Generation

Creating publication-quality visualizations using colorblind-safe palettes and consistent design principles. Each visualization addresses specific analytical questions about global TB burden patterns.

In [ ]:
def create_choropleth_map(df: pd.DataFrame, year: int = None) -> go.Figure:
    """
    Create global choropleth map of TB incidence rates
    """
    if year is None:
        year = df['year'].max()
    
    # Filter data for the specified year
    df_year = df[df['year'] == year].copy()
    
    # Handle missing values
    df_year = df_year.dropna(subset=['incidence_per100k', 'iso3'])
    
    fig = px.choropleth(
        df_year,
        locations='iso3',
        color='incidence_per100k',
        hover_name='country',
        hover_data={
            'incidence_per100k': ':.1f',
            'population': ':,',
            'year': True,
            'iso3': False
        },
        color_continuous_scale='Viridis',
        labels={'incidence_per100k': 'TB Incidence per 100k'},
        title=f'Global TB Incidence Rates per 100,000 Population ({year})',
        projection='natural earth'
    )
    
    fig.update_layout(
        title_font_size=20,
        title_x=0.5,
        geo=dict(showframe=False, showcoastlines=True),
        coloraxis_colorbar=dict(
            title="Cases per<br>100,000",
            title_font_size=12,
            tickfont_size=10
        ),
        width=1200,
        height=700
    )
    
    # Save figure
    fig.write_image(f'figures/choropleth_incidence_per100k_{year}.png', scale=3)
    print(f"✓ Saved choropleth map: figures/choropleth_incidence_per100k_{year}.png")
    
    return fig

# Create and display choropleth map
latest_year = df_clean['year'].max()
fig1 = create_choropleth_map(df_clean, latest_year)
fig1.show()

### Global TB Incidence Distribution

The choropleth map reveals stark geographical disparities in TB burden, with Sub-Saharan Africa and parts of Asia showing the highest incidence rates. This visualization enables immediate identification of priority regions requiring intensive intervention strategies.

In [ ]:
def create_top10_bar_chart(df: pd.DataFrame, year: int = None) -> go.Figure:
    """
    Create horizontal bar chart of top 10 countries by TB incidence rate
    """
    if year is None:
        year = df['year'].max()
    
    # Filter and prepare data
    df_year = df[df['year'] == year].copy()
    df_year = df_year.dropna(subset=['incidence_per100k', 'country'])
    
    # Get top 10 countries
    top10 = df_year.nlargest(10, 'incidence_per100k')
    
    # Calculate 5-year change if available
    year_5_ago = year - 5
    if year_5_ago in df['year'].values:
        df_5_ago = df[df['year'] == year_5_ago][['country', 'incidence_per100k']]
        df_5_ago = df_5_ago.rename(columns={'incidence_per100k': 'incidence_5_ago'})
        top10 = top10.merge(df_5_ago, on='country', how='left')
        top10['pct_change_5yr'] = ((top10['incidence_per100k'] - top10['incidence_5_ago']) / 
                                  top10['incidence_5_ago'] * 100)
    else:
        top10['pct_change_5yr'] = np.nan
    
    # Create horizontal bar chart
    fig = px.bar(
        top10.sort_values('incidence_per100k'),
        x='incidence_per100k',
        y='country',
        orientation='h',
        color='incidence_per100k',
        color_continuous_scale='Viridis',
        title=f'Top 10 Countries by TB Incidence Rate ({year})',
        labels={'incidence_per100k': 'TB Incidence per 100,000 Population'}
    )
    
    # Add annotations for values and 5-year change
    for i, (_, row) in enumerate(top10.sort_values('incidence_per100k').iterrows()):
        annotation_text = f"{row['incidence_per100k']:.0f}"
        if not np.isnan(row.get('pct_change_5yr', np.nan)):
            change_sign = "+" if row['pct_change_5yr'] > 0 else ""
            annotation_text += f"<br>({change_sign}{row['pct_change_5yr']:.1f}% vs {year_5_ago})"
        
        fig.add_annotation(
            x=row['incidence_per100k'],
            y=i,
            text=annotation_text,
            showarrow=False,
            xanchor='left',
            font=dict(size=10, color='white' if row['incidence_per100k'] > top10['incidence_per100k'].median() else 'black')
        )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12,
        showlegend=False,
        width=1000,
        height=600,
        margin=dict(l=150, r=50, t=80, b=50)
    )
    
    # Save figure
    fig.write_image(f'figures/top10_incidence_per100k_{year}.png', scale=3)
    print(f"✓ Saved top 10 bar chart: figures/top10_incidence_per100k_{year}.png")
    
    return fig

# Create and display top 10 bar chart
fig2 = create_top10_bar_chart(df_clean, latest_year)
fig2.show()

### Highest Burden Countries

The horizontal bar chart highlights countries with the most severe TB burden per capita, enabling comparison of 5-year trends where data is available. This ranking helps prioritize resource allocation and intervention strategies.

In [ ]:
def create_trend_analysis(df: pd.DataFrame) -> go.Figure:
    """
    Create multi-line trend chart for top 5 countries by recent incidence
    """
    latest_year = df['year'].max()
    
    # Get top 5 countries by latest year incidence
    df_latest = df[df['year'] == latest_year].dropna(subset=['incidence_per100k', 'country'])
    top5_countries = df_latest.nlargest(5, 'incidence_per100k')['country'].tolist()
    
    # Filter data for top 5 countries
    df_trends = df[df['country'].isin(top5_countries)].copy()
    df_trends = df_trends.dropna(subset=['incidence_per100k', 'year'])
    df_trends = df_trends.sort_values(['country', 'year'])
    
    # Create line chart
    fig = px.line(
        df_trends,
        x='year',
        y='incidence_per100k',
        color='country',
        title='TB Incidence Trends: Top 5 Countries',
        labels={
            'incidence_per100k': 'TB Incidence per 100,000 Population',
            'year': 'Year',
            'country': 'Country'
        },
        color_discrete_sequence=px.colors.qualitative.Set1
    )
    
    # Add start and end point annotations
    for country in top5_countries:
        country_data = df_trends[df_trends['country'] == country]
        if len(country_data) > 0:
            start_data = country_data.iloc[0]
            end_data = country_data.iloc[-1]
            
            # Start point annotation
            fig.add_annotation(
                x=start_data['year'],
                y=start_data['incidence_per100k'],
                text=f"{start_data['incidence_per100k']:.0f}",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=1,
                arrowcolor="gray",
                font=dict(size=10)
            )
            
            # End point annotation
            fig.add_annotation(
                x=end_data['year'],
                y=end_data['incidence_per100k'],
                text=f"{end_data['incidence_per100k']:.0f}",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=1,
                arrowcolor="gray",
                font=dict(size=10)
            )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        width=1000,
        height=600
    )
    
    fig.update_traces(line=dict(width=3), marker=dict(size=6))
    
    # Save figure
    fig.write_image('figures/trends_top5.png', scale=3)
    print("✓ Saved trend analysis: figures/trends_top5.png")
    
    return fig

# Create and display trend analysis
fig3 = create_trend_analysis(df_clean)
fig3.show()

### Temporal Trends Analysis  

The multi-line trend chart reveals divergent trajectories among high-burden countries, with some showing declining trends while others remain stable or increasing. Start and end point annotations facilitate quick assessment of progress over the observation period.